In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import optuna
import os

# Load the data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Separate features and target
X = train.drop(columns=["price_doc", "id"], errors='ignore')
y = train["price_doc"]

# Handle Outliers (Example: Remove extreme price outliers)
q1 = y.quantile(0.01)
q99 = y.quantile(0.99)
outlier_mask = (y >= q1) & (y <= q99)
X = X[outlier_mask]
y = y[outlier_mask]

# Impute missing values
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Numerical imputation
num_imputer = SimpleImputer(strategy='median')
X[numerical_features] = num_imputer.fit_transform(X[numerical_features])
test[numerical_features] = num_imputer.transform(test[numerical_features])

# Categorical encoding and imputation
cat_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_features] = cat_imputer.fit_transform(X[categorical_features])
test[categorical_features] = cat_imputer.transform(test[categorical_features])

label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    test[col] = le.transform(test[col])
    label_encoders[col] = le

# Feature Scaling
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])
test[numerical_features] = scaler.transform(test[numerical_features])

# XGBoost for Feature Selection
xgb_fs_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_jobs=-1,
    random_state=42,
    n_estimators=100
)
xgb_fs_model.fit(X, y)

# Get feature importance scores
feature_importances = pd.Series(xgb_fs_model.feature_importances_, index=X.columns)

# Define a function to evaluate the number of top features
def evaluate_top_k_features(k):
    top_features = feature_importances.nlargest(k).index
    X_k = X[top_features]
    X_train, X_val, y_train, y_val = train_test_split(X_k, y, test_size=0.2, random_state=42)

    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
            "max_depth": trial.suggest_int("max_depth", 3, 15),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.05),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 5),
            "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
            "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
        }
        model = xgb.XGBRegressor(
            objective="reg:squarederror", tree_method="hist", random_state=42, **params
        )
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        return rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=30)

    return study.best_params, study.best_value

# Evaluate different top-k features
best_k = None
best_rmse = float("inf")
best_model = None
test_predictions_best = None

for k in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:  # Test different numbers of features
    print(f"Evaluating top-{k} features...")
    params, rmse = evaluate_top_k_features(k)
    print(f"Top-{k} features: Best RMSE = {rmse:.4f} with params = {params}")
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_k = k
        # Train the model with the best parameters on all data
        top_features = feature_importances.nlargest(k).index
        X_train_full = X[top_features]
        X_train, X_val, y_train, y_val = train_test_split(X_train_full, y, test_size=0.2, random_state=42)

        best_model = xgb.XGBRegressor(
            objective="reg:squarederror", tree_method="hist", random_state=42, **params
        )
        best_model.fit(X_train_full, y)
        test_predictions_best = best_model.predict(test[top_features])

# Save the final submission file using the sample_submission structure
final_submission = sample_submission.copy()
final_submission["price_doc"] = test_predictions_best

# Prepare Submission Directory
if not os.path.exists("submissions5"):
    os.makedirs("submissions5")

# Save the final submission file
final_submission_file = f"submissions5/final_submission_top_{best_k}_features_rmse_{best_rmse:.4f}.csv"
final_submission.to_csv(final_submission_file, index=False)

print(f"Final submission file saved as {final_submission_file}")


C:\Users\aliri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-11-26 11:19:06,759] A new study created in memory with name: no-name-ac9c236b-cc41-4c46-b2ca-64355bed7c47


Evaluating top-10 features...


[I 2024-11-26 11:19:39,097] Trial 0 finished with value: 11853122.48945553 and parameters: {'n_estimators': 723, 'max_depth': 14, 'learning_rate': 0.04240300716253512, 'subsample': 0.698879965625048, 'colsample_bytree': 0.850835095080857, 'gamma': 3.7820923127655064, 'reg_alpha': 0.2949933753586558, 'reg_lambda': 0.1496604493706618}. Best is trial 0 with value: 11853122.48945553.
[I 2024-11-26 11:19:39,989] Trial 1 finished with value: 11712574.575729195 and parameters: {'n_estimators': 273, 'max_depth': 3, 'learning_rate': 0.047936228341382975, 'subsample': 0.9241106695685668, 'colsample_bytree': 0.745871238421505, 'gamma': 0.786446555149708, 'reg_alpha': 0.40607833231843826, 'reg_lambda': 0.6455466002430947}. Best is trial 1 with value: 11712574.575729195.
[I 2024-11-26 11:19:42,544] Trial 2 finished with value: 11798499.169527147 and parameters: {'n_estimators': 466, 'max_depth': 7, 'learning_rate': 0.028620497414232022, 'subsample': 0.6755251961650299, 'colsample_bytree': 0.9306006

Top-10 features: Best RMSE = 11623925.9813 with params = {'n_estimators': 538, 'max_depth': 14, 'learning_rate': 0.006171866168655084, 'subsample': 0.7624987554572065, 'colsample_bytree': 0.8067352944959315, 'gamma': 4.0747682395302025, 'reg_alpha': 0.8642525133472556, 'reg_lambda': 0.5285025509778825}


[I 2024-11-26 11:25:07,825] A new study created in memory with name: no-name-077c648b-f0ee-46bc-97c5-01cbf5155ae6


...

Top-80 features: Best RMSE = 11560115.4433 with params = {'n_estimators': 678, 'max_depth': 15, 'learning_rate': 0.008974509844067908, 'subsample': 0.7898094774202821, 'colsample_bytree': 0.9034302916337132, 'gamma': 2.408130519984519, 'reg_alpha': 0.91954931048039, 'reg_lambda': 0.597416762454551}
Evaluating top-90 features...


[I 2024-11-26 14:34:35,979] A new study created in memory with name: no-name-07077c68-2b5d-4f2f-83e8-0b6c270b5ab6
[I 2024-11-26 14:35:02,813] Trial 0 finished with value: 12109369.657416519 and parameters: {'n_estimators': 267, 'max_depth': 11, 'learning_rate': 0.005702824700373133, 'subsample': 0.6046103431736545, 'colsample_bytree': 0.8438351234688313, 'gamma': 2.5337193789777697, 'reg_alpha': 0.026174429011803224, 'reg_lambda': 0.8462665610430956}. Best is trial 0 with value: 12109369.657416519.
[I 2024-11-26 14:35:33,107] Trial 1 finished with value: 11712230.33618503 and parameters: {'n_estimators': 218, 'max_depth': 12, 'learning_rate': 0.04237174535160784, 'subsample': 0.6745853153586069, 'colsample_bytree': 0.9862343564482955, 'gamma': 1.8291296013587022, 'reg_alpha': 0.19681601083316969, 'reg_lambda': 0.5736716615467293}. Best is trial 1 with value: 11712230.33618503.
[I 2024-11-26 14:36:54,236] Trial 2 finished with value: 11635101.720193718 and parameters: {'n_estimators': 5

Top-90 features: Best RMSE = 11557940.0108 with params = {'n_estimators': 370, 'max_depth': 14, 'learning_rate': 0.010213130303822225, 'subsample': 0.6765603269622176, 'colsample_bytree': 0.9189316792044802, 'gamma': 3.6780229374628495, 'reg_alpha': 0.6409528347226071, 'reg_lambda': 0.31479542014006545}
Evaluating top-100 features...


[I 2024-11-26 15:12:10,743] A new study created in memory with name: no-name-76ed4ffa-2753-4922-8e34-366c3d3bef29
[I 2024-11-26 15:13:34,868] Trial 0 finished with value: 11657826.47039754 and parameters: {'n_estimators': 288, 'max_depth': 14, 'learning_rate': 0.0447945815278724, 'subsample': 0.7173322182423058, 'colsample_bytree': 0.8221114304827797, 'gamma': 4.279839980691496, 'reg_alpha': 0.08456985361237912, 'reg_lambda': 0.02459865739905387}. Best is trial 0 with value: 11657826.47039754.
[I 2024-11-26 15:13:43,636] Trial 1 finished with value: 11667521.27685338 and parameters: {'n_estimators': 630, 'max_depth': 5, 'learning_rate': 0.011411909214257668, 'subsample': 0.6261826269733218, 'colsample_bytree': 0.8381039760942728, 'gamma': 1.4235459829204278, 'reg_alpha': 0.5007518136644165, 'reg_lambda': 0.22664618710067752}. Best is trial 0 with value: 11657826.47039754.
[I 2024-11-26 15:14:20,350] Trial 2 finished with value: 11629668.432691434 and parameters: {'n_estimators': 576, '

Top-100 features: Best RMSE = 11510181.6748 with params = {'n_estimators': 638, 'max_depth': 13, 'learning_rate': 0.009032888165546774, 'subsample': 0.9591515613470262, 'colsample_bytree': 0.6844171793158738, 'gamma': 4.87906163439442, 'reg_alpha': 0.40782446819729457, 'reg_lambda': 0.2841053248975898}
Final submission file saved as submissions5/final_submission_top_100_features_rmse_11510181.6748.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import optuna
import os

# Load the data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Separate features and target
X = train.drop(columns=["price_doc", "id"], errors='ignore')
y = train["price_doc"]

# Handle Outliers (Example: Remove extreme price outliers)
q1 = y.quantile(0.01)
q99 = y.quantile(0.99)
outlier_mask = (y >= q1) & (y <= q99)
X = X[outlier_mask]
y = y[outlier_mask]

# Impute missing values
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Numerical imputation
num_imputer = SimpleImputer(strategy='median')
X[numerical_features] = num_imputer.fit_transform(X[numerical_features])
test[numerical_features] = num_imputer.transform(test[numerical_features])

# Categorical encoding and imputation
cat_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_features] = cat_imputer.fit_transform(X[categorical_features])
test[categorical_features] = cat_imputer.transform(test[categorical_features])

label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    test[col] = le.transform(test[col])
    label_encoders[col] = le

# Feature Scaling
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])
test[numerical_features] = scaler.transform(test[numerical_features])

# XGBoost for Feature Selection
xgb_fs_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_jobs=-1,
    random_state=42,
    n_estimators=100
)
xgb_fs_model.fit(X, y)

# Get feature importance scores
feature_importances = pd.Series(xgb_fs_model.feature_importances_, index=X.columns)

# Define a function to evaluate the number of top features
def evaluate_top_k_features(k):
    top_features = feature_importances.nlargest(k).index
    X_k = X[top_features]
    X_train, X_val, y_train, y_val = train_test_split(X_k, y, test_size=0.2, random_state=42)

    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
            "max_depth": trial.suggest_int("max_depth", 3, 15),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.05),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 5),
            "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
            "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
        }
        model = xgb.XGBRegressor(
            objective="reg:squarederror", tree_method="hist", random_state=42, **params
        )
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        return rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=30)

    return study.best_params, study.best_value

# Evaluate different top-k features
best_k = None
best_rmse = float("inf")
best_model = None
test_predictions_best = None

for k in [110, 120, 130, 140, 150, 160, 170, 180, 190, 200]:  # Test different numbers of features
    print(f"Evaluating top-{k} features...")
    params, rmse = evaluate_top_k_features(k)
    print(f"Top-{k} features: Best RMSE = {rmse:.4f} with params = {params}")
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_k = k
        # Train the model with the best parameters on all data
        top_features = feature_importances.nlargest(k).index
        X_train_full = X[top_features]
        X_train, X_val, y_train, y_val = train_test_split(X_train_full, y, test_size=0.2, random_state=42)

        best_model = xgb.XGBRegressor(
            objective="reg:squarederror", tree_method="hist", random_state=42, **params
        )
        best_model.fit(X_train_full, y)
        test_predictions_best = best_model.predict(test[top_features])

# Save the final submission file using the sample_submission structure
final_submission = sample_submission.copy()
final_submission["price_doc"] = test_predictions_best

# Prepare Submission Directory
if not os.path.exists("submissions5"):
    os.makedirs("submissions5")

# Save the final submission file
final_submission_file = f"submissions5/final_submission_top_{best_k}_features_rmse_{best_rmse:.4f}.csv"
final_submission.to_csv(final_submission_file, index=False)

print(f"Final submission file saved as {final_submission_file}")


Evaluating top-110 features...


[I 2024-11-26 17:44:12,103] A new study created in memory with name: no-name-8ca6ff2b-554c-4ef7-bd51-df2777f80649
[I 2024-11-26 17:45:30,412] Trial 0 finished with value: 11562127.074038265 and parameters: {'n_estimators': 373, 'max_depth': 13, 'learning_rate': 0.016135069206283093, 'subsample': 0.969509695908804, 'colsample_bytree': 0.6449701497452142, 'gamma': 4.762292087863329, 'reg_alpha': 0.05642548077680365, 'reg_lambda': 0.2940976783748539}. Best is trial 0 with value: 11562127.074038265.
[I 2024-11-26 17:47:40,959] Trial 1 finished with value: 11678641.720181199 and parameters: {'n_estimators': 512, 'max_depth': 14, 'learning_rate': 0.0366336098021867, 'subsample': 0.6898289423523121, 'colsample_bytree': 0.739817455197801, 'gamma': 3.650466701483353, 'reg_alpha': 0.5210372178060307, 'reg_lambda': 0.2927125754594764}. Best is trial 0 with value: 11562127.074038265.
[I 2024-11-26 17:49:28,055] Trial 2 finished with value: 11545626.993324464 and parameters: {'n_estimators': 249, '

Top-110 features: Best RMSE = 11505893.1348 with params = {'n_estimators': 966, 'max_depth': 14, 'learning_rate': 0.005433020622158103, 'subsample': 0.9467626338208265, 'colsample_bytree': 0.6653981017646022, 'gamma': 4.5147629818758155, 'reg_alpha': 0.08064963132741576, 'reg_lambda': 0.1282605544799642}
Evaluating top-120 features...


[I 2024-11-26 18:46:43,783] A new study created in memory with name: no-name-7ab2e142-2ea3-4075-9beb-e6a9e29a74d8
[I 2024-11-26 18:46:51,810] Trial 0 finished with value: 11711270.137047278 and parameters: {'n_estimators': 750, 'max_depth': 3, 'learning_rate': 0.022868730504128937, 'subsample': 0.7131948228705065, 'colsample_bytree': 0.7504294905672823, 'gamma': 2.138752833816548, 'reg_alpha': 0.7333531217368262, 'reg_lambda': 0.01278973129440275}. Best is trial 0 with value: 11711270.137047278.
[I 2024-11-26 18:47:29,966] Trial 1 finished with value: 11588387.91966546 and parameters: {'n_estimators': 630, 'max_depth': 9, 'learning_rate': 0.008920122662599297, 'subsample': 0.9136108042778269, 'colsample_bytree': 0.9322696785528558, 'gamma': 2.485741585850434, 'reg_alpha': 0.5650388636154775, 'reg_lambda': 0.6731036809007764}. Best is trial 1 with value: 11588387.91966546.
[I 2024-11-26 18:49:00,784] Trial 2 finished with value: 11661431.27568049 and parameters: {'n_estimators': 792, 'm

Top-120 features: Best RMSE = 11512710.1295 with params = {'n_estimators': 589, 'max_depth': 13, 'learning_rate': 0.00632400532912893, 'subsample': 0.8875038586752022, 'colsample_bytree': 0.8076848572010236, 'gamma': 0.3888453931576725, 'reg_alpha': 0.79702474696123, 'reg_lambda': 0.01984297155665038}
Evaluating top-130 features...


...

Top-180 features: Best RMSE = 11504912.6203 with params = {'n_estimators': 986, 'max_depth': 14, 'learning_rate': 0.00452717139253509, 'subsample': 0.6850521380601174, 'colsample_bytree': 0.702550953976305, 'gamma': 0.20964625840740525, 'reg_alpha': 0.06948269856327328, 'reg_lambda': 0.4165429364862752}
Evaluating top-190 features...


[I 2024-11-27 03:25:59,180] A new study created in memory with name: no-name-47fb3ac3-d90f-4d79-b6ae-3eefc07a8d17
[I 2024-11-27 03:27:06,277] Trial 0 finished with value: 11661760.029440949 and parameters: {'n_estimators': 558, 'max_depth': 10, 'learning_rate': 0.04253876794680194, 'subsample': 0.893568757236596, 'colsample_bytree': 0.7921818019109343, 'gamma': 3.3312768849900736, 'reg_alpha': 0.13351225017009982, 'reg_lambda': 0.6016886744997723}. Best is trial 0 with value: 11661760.029440949.
[I 2024-11-27 03:28:36,617] Trial 1 finished with value: 17455942.698841203 and parameters: {'n_estimators': 152, 'max_depth': 15, 'learning_rate': 0.0015153912112042468, 'subsample': 0.6106346771251367, 'colsample_bytree': 0.6018562073765931, 'gamma': 2.182969146002536, 'reg_alpha': 0.10508613858973248, 'reg_lambda': 0.8190393411434175}. Best is trial 0 with value: 11661760.029440949.
[I 2024-11-27 03:29:00,799] Trial 2 finished with value: 11728192.08059442 and parameters: {'n_estimators': 78

Top-190 features: Best RMSE = 11476342.2368 with params = {'n_estimators': 725, 'max_depth': 15, 'learning_rate': 0.010226887130093398, 'subsample': 0.8140286256566845, 'colsample_bytree': 0.6874697388201212, 'gamma': 0.7388217185974744, 'reg_alpha': 0.7715128316223478, 'reg_lambda': 0.7549000911002367}
Evaluating top-200 features...


[I 2024-11-27 05:12:35,853] A new study created in memory with name: no-name-e213ffee-c6b4-48eb-b5f9-708fd2b139b0
[I 2024-11-27 05:12:42,729] Trial 0 finished with value: 11706737.658791846 and parameters: {'n_estimators': 299, 'max_depth': 3, 'learning_rate': 0.016327814416724908, 'subsample': 0.7239029541104568, 'colsample_bytree': 0.7065262257962206, 'gamma': 2.877378032717828, 'reg_alpha': 0.033263290188197736, 'reg_lambda': 0.3391454411609095}. Best is trial 0 with value: 11706737.658791846.
[I 2024-11-27 05:12:55,983] Trial 1 finished with value: 13831569.849037213 and parameters: {'n_estimators': 515, 'max_depth': 4, 'learning_rate': 0.0015560439300566606, 'subsample': 0.9941370405898702, 'colsample_bytree': 0.9951990603648336, 'gamma': 3.458526056447699, 'reg_alpha': 0.03626156006787007, 'reg_lambda': 0.5034886778170442}. Best is trial 0 with value: 11706737.658791846.
[I 2024-11-27 05:13:07,347] Trial 2 finished with value: 11724406.658323942 and parameters: {'n_estimators': 5

Top-200 features: Best RMSE = 11523689.8968 with params = {'n_estimators': 611, 'max_depth': 14, 'learning_rate': 0.02673119351299593, 'subsample': 0.9565736188062032, 'colsample_bytree': 0.8200301399481642, 'gamma': 2.3803458355586997, 'reg_alpha': 0.8391484206455042, 'reg_lambda': 0.8275267768741771}
Final submission file saved as submissions5/final_submission_top_190_features_rmse_11476342.2368.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import optuna
import os

# Load the data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Separate features and target
X = train.drop(columns=["price_doc", "id"], errors='ignore')
y = train["price_doc"]

# Handle Outliers (Example: Remove extreme price outliers)
q1 = y.quantile(0.01)
q99 = y.quantile(0.99)
outlier_mask = (y >= q1) & (y <= q99)
X = X[outlier_mask]
y = y[outlier_mask]

# Impute missing values
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Numerical imputation
num_imputer = SimpleImputer(strategy='median')
X[numerical_features] = num_imputer.fit_transform(X[numerical_features])
test[numerical_features] = num_imputer.transform(test[numerical_features])

# Categorical encoding and imputation
cat_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_features] = cat_imputer.fit_transform(X[categorical_features])
test[categorical_features] = cat_imputer.transform(test[categorical_features])

label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    test[col] = le.transform(test[col])
    label_encoders[col] = le

# Feature Scaling
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])
test[numerical_features] = scaler.transform(test[numerical_features])

# XGBoost for Feature Selection
xgb_fs_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_jobs=-1,
    random_state=42,
    n_estimators=100
)
xgb_fs_model.fit(X, y)

# Get feature importance scores
feature_importances = pd.Series(xgb_fs_model.feature_importances_, index=X.columns)

# Define a function to evaluate the number of top features
def evaluate_top_k_features(k):
    top_features = feature_importances.nlargest(k).index
    X_k = X[top_features]
    X_train, X_val, y_train, y_val = train_test_split(X_k, y, test_size=0.2, random_state=42)

    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
            "max_depth": trial.suggest_int("max_depth", 3, 15),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.05),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 5),
            "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
            "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
        }
        model = xgb.XGBRegressor(
            objective="reg:squarederror", tree_method="hist", random_state=42, **params
        )
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        return rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=30)

    return study.best_params, study.best_value

# Evaluate different top-k features
best_k = None
best_rmse = float("inf")
best_model = None
test_predictions_best = None

for k in [210, 220, 230, 240, 250, 260]:  # Test different numbers of features
    print(f"Evaluating top-{k} features...")
    params, rmse = evaluate_top_k_features(k)
    print(f"Top-{k} features: Best RMSE = {rmse:.4f} with params = {params}")
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_k = k
        # Train the model with the best parameters on all data
        top_features = feature_importances.nlargest(k).index
        X_train_full = X[top_features]
        X_train, X_val, y_train, y_val = train_test_split(X_train_full, y, test_size=0.2, random_state=42)

        best_model = xgb.XGBRegressor(
            objective="reg:squarederror", tree_method="hist", random_state=42, **params
        )
        best_model.fit(X_train_full, y)
        test_predictions_best = best_model.predict(test[top_features])

# Save the final submission file using the sample_submission structure
final_submission = sample_submission.copy()
final_submission["price_doc"] = test_predictions_best

# Prepare Submission Directory
if not os.path.exists("submissions5"):
    os.makedirs("submissions5")

# Save the final submission file
final_submission_file = f"submissions5/final_submission_top_{best_k}_features_rmse_{best_rmse:.4f}.csv"
final_submission.to_csv(final_submission_file, index=False)

print(f"Final submission file saved as {final_submission_file}")


Evaluating top-210 features...


[I 2024-11-27 09:02:49,560] A new study created in memory with name: no-name-d383ef31-26f1-4c7d-bdcb-03332a886e25
[I 2024-11-27 09:02:55,826] Trial 0 finished with value: 11705457.488878509 and parameters: {'n_estimators': 179, 'max_depth': 5, 'learning_rate': 0.015699759609221233, 'subsample': 0.6870732612041776, 'colsample_bytree': 0.6944747539902663, 'gamma': 1.5118248994991235, 'reg_alpha': 0.43378349618444456, 'reg_lambda': 0.1810878949457877}. Best is trial 0 with value: 11705457.488878509.
[I 2024-11-27 09:06:52,704] Trial 1 finished with value: 11594441.658483226 and parameters: {'n_estimators': 419, 'max_depth': 14, 'learning_rate': 0.040982631460510316, 'subsample': 0.9878512347069321, 'colsample_bytree': 0.9269083563529666, 'gamma': 3.6250106019413844, 'reg_alpha': 0.16532307140558145, 'reg_lambda': 0.9971676903728091}. Best is trial 1 with value: 11594441.658483226.
[I 2024-11-27 09:09:05,166] Trial 2 finished with value: 11666555.837646687 and parameters: {'n_estimators': 

Top-210 features: Best RMSE = 11516324.4145 with params = {'n_estimators': 870, 'max_depth': 13, 'learning_rate': 0.03372867302760106, 'subsample': 0.9342355661317092, 'colsample_bytree': 0.8757942561491007, 'gamma': 4.015532119062435, 'reg_alpha': 0.6141549709651437, 'reg_lambda': 0.25872728482560003}
Evaluating top-220 features...


[I 2024-11-27 11:01:22,687] A new study created in memory with name: no-name-eef9fbe9-b9f3-4ab9-b1d4-143421f241dd
[I 2024-11-27 11:01:43,937] Trial 0 finished with value: 11672064.427503519 and parameters: {'n_estimators': 871, 'max_depth': 4, 'learning_rate': 0.010308075066636693, 'subsample': 0.7131997041023364, 'colsample_bytree': 0.7754169791262724, 'gamma': 3.076628548944322, 'reg_alpha': 0.3426111946321627, 'reg_lambda': 0.6868532729399124}. Best is trial 0 with value: 11672064.427503519.
[I 2024-11-27 11:11:29,636] Trial 1 finished with value: 11491328.481578592 and parameters: {'n_estimators': 926, 'max_depth': 14, 'learning_rate': 0.004479816749385142, 'subsample': 0.8361588804189354, 'colsample_bytree': 0.846621400035282, 'gamma': 1.4928382763365577, 'reg_alpha': 0.5872981384085251, 'reg_lambda': 0.10348588777796464}. Best is trial 1 with value: 11491328.481578592.
[I 2024-11-27 11:11:46,978] Trial 2 finished with value: 11597019.501777604 and parameters: {'n_estimators': 148

Top-220 features: Best RMSE = 11491328.4816 with params = {'n_estimators': 926, 'max_depth': 14, 'learning_rate': 0.004479816749385142, 'subsample': 0.8361588804189354, 'colsample_bytree': 0.846621400035282, 'gamma': 1.4928382763365577, 'reg_alpha': 0.5872981384085251, 'reg_lambda': 0.10348588777796464}
Evaluating top-230 features...


...

Top-240 features: Best RMSE = 11527835.1995 with params = {'n_estimators': 995, 'max_depth': 12, 'learning_rate': 0.006686795095652565, 'subsample': 0.8260902071130293, 'colsample_bytree': 0.6702608157030406, 'gamma': 3.8034219603226456, 'reg_alpha': 0.42702112101980466, 'reg_lambda': 0.17762395969900624}
Evaluating top-250 features...


[I 2024-11-27 16:37:22,160] A new study created in memory with name: no-name-42e68bca-da17-4d1f-92b4-1bb878473419
[I 2024-11-27 16:37:39,196] Trial 0 finished with value: 11652229.806747235 and parameters: {'n_estimators': 466, 'max_depth': 5, 'learning_rate': 0.009999320308035027, 'subsample': 0.8902691899358455, 'colsample_bytree': 0.7785017805547635, 'gamma': 0.25860206794787677, 'reg_alpha': 0.7637957729280413, 'reg_lambda': 0.44978705299727084}. Best is trial 0 with value: 11652229.806747235.
[I 2024-11-27 16:37:46,893] Trial 1 finished with value: 11658243.315904103 and parameters: {'n_estimators': 179, 'max_depth': 5, 'learning_rate': 0.027573309128358956, 'subsample': 0.8629383588535193, 'colsample_bytree': 0.7260362915345069, 'gamma': 2.5616085500343164, 'reg_alpha': 0.6277860233717247, 'reg_lambda': 0.8977698764009324}. Best is trial 0 with value: 11652229.806747235.
[I 2024-11-27 16:38:17,462] Trial 2 finished with value: 11750205.44419666 and parameters: {'n_estimators': 73

Top-250 features: Best RMSE = 11510927.9200 with params = {'n_estimators': 247, 'max_depth': 15, 'learning_rate': 0.01780488757040003, 'subsample': 0.7744864530937439, 'colsample_bytree': 0.9971584251588652, 'gamma': 1.6158704147017455, 'reg_alpha': 0.9773849216361268, 'reg_lambda': 0.30519995887239276}
Evaluating top-260 features...


[I 2024-11-27 17:41:44,272] A new study created in memory with name: no-name-048147cd-d326-4870-9a6c-4988df1480d7
[I 2024-11-27 17:43:39,954] Trial 0 finished with value: 11634677.131602582 and parameters: {'n_estimators': 219, 'max_depth': 13, 'learning_rate': 0.03513357254453264, 'subsample': 0.6730468454203806, 'colsample_bytree': 0.9380941522653625, 'gamma': 0.22208876937933886, 'reg_alpha': 0.7568062907191622, 'reg_lambda': 0.1641728120816286}. Best is trial 0 with value: 11634677.131602582.
[I 2024-11-27 17:53:36,894] Trial 1 finished with value: 11496239.623931287 and parameters: {'n_estimators': 723, 'max_depth': 15, 'learning_rate': 0.01348227576757648, 'subsample': 0.6896765016596557, 'colsample_bytree': 0.9765448577986259, 'gamma': 0.596140543603319, 'reg_alpha': 0.8722264645148121, 'reg_lambda': 0.44658648035967174}. Best is trial 1 with value: 11496239.623931287.
[I 2024-11-27 17:54:06,685] Trial 2 finished with value: 11666212.874075463 and parameters: {'n_estimators': 99

Top-260 features: Best RMSE = 11496239.6239 with params = {'n_estimators': 723, 'max_depth': 15, 'learning_rate': 0.01348227576757648, 'subsample': 0.6896765016596557, 'colsample_bytree': 0.9765448577986259, 'gamma': 0.596140543603319, 'reg_alpha': 0.8722264645148121, 'reg_lambda': 0.44658648035967174}
Final submission file saved as submissions5/final_submission_top_220_features_rmse_11491328.4816.csv
